# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.


### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')


### Answers to Q1 and Q2 

+ We can test this using t-test. These are clearly binomial distributions. Since the researchers randomly assigned identical résumés black-sounding or white-sounding names to employers and the sample is sufficiently large (n > 30). Also, There are at least 10 successes and 10 failures in the sample. We can definitely apply CLT in this case.
+ Null hypothesis is race has no significant impact on the rate of callbacks for resumes.
    <br / >Alternate hypothesis is race has a significant impact on the rate of callbacks for resumes. That's black sounding names get a significantly lower callback rate than white sounding names.
    
 

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [5]:
success = len(data[data.call==1])/len(data)
total = len(data)
min(success*total, (1-success)*total) > 10

True

In [6]:
w = data[data.race=='w']
b = data[data.race=='b']

### Answers to Q3

In [7]:
success_b= len(b[b.call==1])/len(b)
success_w= len(w[w.call==1])/len(w)
diff_mean = success_w - success_b
diff_mean

0.032032854209445585

In [10]:
# #For a 95% confidence interval, use a z* = 1.96. The margin of error (ME) and confidence interval (CI) are:
# bstd = success_b*(1-success_b)
# wstd = success_w*(1-success_w)
# margin = 1.96 * (np.sqrt(bstd* bstd/len(b)) + wstd* wstd/len(w))
# CI = [diff_mean-margin, diff_mean+margin]
# print('confidence interval :', CI)
# # np.sqrt(bstd* bstd/len(b)) + wstd* wstd/len(w)

In [11]:
#For a 95% confidence interval, use a z* = 1.96. The margin of error (ME) and confidence interval (CI) are:
std = success*(1-success)
margin = 1.96 * (np.sqrt(1/len(b) + 1/len(w)))* std
CI = [diff_mean-margin, diff_mean+margin]
print('confidence interval :', CI)

confidence interval : [0.027875339252990638, 0.036190369165900529]



### Answers to Q4 and Q5 here
+ Since p_value is 0 we can reject the null which means black sounding names get a significantly lower callback rate than white sounding names.
+ In this analysis, it doesn't mean race/name is the most important factor in callback success, it's just one of the factors. In the future work, I can use logistic regression to take every columns into consideration which may be more appropriate.

In [36]:
# calculate sample probability of sucess (being called back) for a white/black person
rate_w = len(w[w.call==1])/len(w)
rate_b = len(b[b.call==1])/len(b)
print("rate_w :", rate_w)
print("rate_b :", rate_b)
print("rate_pop :", success)

rate_w : 0.09650924024640657
rate_b : 0.06447638603696099
rate_pop : 0.08049281314168377


In [37]:
#Draw 10,000 permutation replicates and compute the p-value
black = list(b.call)
white = list(w.call)
pop = list(data.call)
bs_replicates = np.empty(10000)
for i in range(len(bs_replicates)):
    permuted = np.random.permutation(pop)
    perm_b = permuted[:len(black)]
    bs_replicates[i] = np.mean(perm_b)
p = np.sum(bs_replicates <= rate_b) / len(bs_replicates)
print('p-value =', p)

p-value = 0.0
